<a href="https://colab.research.google.com/github/faruq2021/My-ML-MODEL-NOTEBOOKS/blob/main/attainhelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
table=pd.read_csv("/content/dbo-user_roles-121-live.1690887165.csv")

In [6]:
table.head(20)

,id,created_at,updated_at,role_id,user_id,organization_id
0,1,1678188102403,NaN,2.0,2.0,1
1,2,1678188748408,NaN,2.0,1.0,1
2,3,1678364291689,NaN,4.0,3.0,1
3,4,1679577582879,NaN,4.0,2.0,1
4,5,1683981876930,NaN,6.0,2.0,1
5,6,1683982999742,NaN,1.0,1.0,1
6,7,1683989517132,NaN,8.0,2.0,1
7,11,1683996515849,NaN,7.0,2.0,1
8,13,1684002096221,NaN,5.0,2.0,1
9,14,1684002898164,NaN,8.0,1.0,1


In [7]:
!pip install transformers
from transformers import TapasTokenizer

# Load the TapasTokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00


In [8]:
# Generate a list of questions for each column and row combination
questions = []
for col in table.columns:
    for i, row in table.iterrows():
        questions.append(f"What is the value in the {col} column for row {i + 1}?")

# Generate corresponding answer coordinates and answer text
answer_coordinates = [[(i, col_idx)] for col_idx in range(len(table.columns)) for i in range(len(table))]
answer_text = [str(table.iat[i, col_idx]) for col_idx in range(len(table.columns)) for i in range(len(table))]


In [9]:
# Tokenize the data
inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)

In [10]:
import torch

class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
from transformers import TapasConfig, TapasForQuestionAnswering, AdamW

# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,

)

# Initialize the model
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,

        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_bias', 'column_output_weights', 'aggregation_classifier.weight', 'output_weights', 'output_bias', 'aggregation_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
output_dir = "/content/saved_models"

# Save the model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/saved_models/tokenizer_config.json',
 '/content/saved_models/special_tokens_map.json',
 '/content/saved_models/vocab.txt',
 '/content/saved_models/added_tokens.json')